#### <font color="red">以下の中から予測したい項目のコメントをはずしてください。</font>

In [0]:
target = '売上高'
# target = '営業利益'
# target = '経常利益'
# target = '税引前純利益'

### 必要なライブラリをインポートします。

In [0]:
import pandas as pd
import numpy as np

### データをダウンロードします。

In [336]:
! wget -O summary-0.csv http://lkzf.info/xbrl/data/2020-04-08/summary-0.csv
! wget -O summary-1.csv http://lkzf.info/xbrl/data/2020-04-08/summary-1.csv
! wget -O summary-2.csv http://lkzf.info/xbrl/data/2020-04-08/summary-2.csv

Will not apply HSTS. The HSTS database must be a regular and non-world-writable file.
ERROR: could not open HSTS store at '/home/hamada/.wget-hsts'. HSTS will be disabled.
--2020-04-07 20:36:24--  http://lkzf.info/xbrl/data/2020-04-07/summary-0.csv
lkzf.info (lkzf.info) をDNSに問いあわせています... 157.7.107.146
lkzf.info (lkzf.info)|157.7.107.146|:80 に接続しています... 接続しました。
HTTP による接続要求を送信しました、応答を待っています... 200 OK
長さ: 4289684 (4.1M) [text/csv]
`summary-0.csv' に保存中

summary-0.csv       100%[===================>]   4.09M  6.32MB/s    時間 0.6s  

2020-04-07 20:36:25 (6.32 MB/s) - `summary-0.csv' へ保存完了 [4289684/4289684]

Will not apply HSTS. The HSTS database must be a regular and non-world-writable file.
ERROR: could not open HSTS store at '/home/hamada/.wget-hsts'. HSTS will be disabled.
--2020-04-07 20:36:26--  http://lkzf.info/xbrl/data/2020-04-07/summary-1.csv
lkzf.info (lkzf.info) をDNSに問いあわせています... 157.7.107.146
lkzf.info (lkzf.info)|157.7.107.146|:80 に接続しています... 接続しました。
HTTP による接続要求を送信しました、応答を待っていま

##### 会社情報をダウンロードします。

In [337]:
! wget -O EdinetcodeDlInfo.csv http://lkzf.info/xbrl/data/EdinetcodeDlInfo.csv

Will not apply HSTS. The HSTS database must be a regular and non-world-writable file.
ERROR: could not open HSTS store at '/home/hamada/.wget-hsts'. HSTS will be disabled.
--2020-04-07 20:37:00--  http://lkzf.info/xbrl/data/EdinetcodeDlInfo.csv
lkzf.info (lkzf.info) をDNSに問いあわせています... 157.7.107.146
lkzf.info (lkzf.info)|157.7.107.146|:80 に接続しています... 接続しました。
HTTP による接続要求を送信しました、応答を待っています... 200 OK
長さ: 1964548 (1.9M) [text/csv]
`EdinetcodeDlInfo.csv' に保存中

EdinetcodeDlInfo.cs 100%[===================>]   1.87M  9.09MB/s    時間 0.2s  

2020-04-07 20:37:01 (9.09 MB/s) - `EdinetcodeDlInfo.csv' へ保存完了 [1964548/1964548]



### CSVファイルの読み込みの関数

In [0]:
def my_read_csv(filename, date_columns=[]):
    """ How to force pandas read_csv to use float32 for all float columns?
    https://stackoverflow.com/questions/30494569/how-to-force-pandas-read-csv-to-use-float32-for-all-float-columns
    """

    # 最初の100行のみ試しに読んでみる。
    df_test = pd.read_csv(filename, nrows=100, parse_dates=date_columns)

    # float64の列の名前
    float_cols = [c for c in df_test if df_test[c].dtype == "float64"]

    # 型をfloat32として宣言する。
    float32_cols = {c: np.float32 for c in float_cols}

    # 型の宣言を使って、CSVファイルを読み直す。
    return pd.read_csv(filename, engine='c', dtype=float32_cols, parse_dates=date_columns)

### 決算書のCSVファイルを読み込みます。

In [339]:
print("read 提出日時点 ...")
df提出日時点 = my_read_csv("summary-0.csv", [ '会計期間終了日' ])
print("read 時点 ...")
df時点 = my_read_csv("summary-1.csv")
print("read 期間 ...")
df期間 = my_read_csv("summary-2.csv")

read 提出日時点 ...
read 時点 ...
read 期間 ...


### 有価証券報告書のみを抜き出します。

In [0]:
df提出日時点 = df提出日時点[ df提出日時点['報告書略号'] == "asr" ]
df時点       = df時点[ df時点['報告書略号'] == "asr" ]
df期間       = df期間[ df期間['報告書略号'] == "asr" ]

for tbl in [ df提出日時点, df時点, df期間 ]:
    tbl.drop('報告書略号', axis=1, inplace=True)

### df時点とdf期間の列で、df提出日時点の列と重複があれば取り除きます。

In [341]:
除外columns = ['index', 'EDINETコード', '会計期間終了日', 'コンテキスト' ]

for tbl in [ df時点, df期間 ]:
    
    # 重複する列の名前のリスト
    dup_columns = [x for x in tbl.columns if x in df提出日時点 and not x in 除外columns]

    if len(dup_columns) != 0:
        # 重複がある場合

        print("重複", dup_columns)
        tbl.drop(dup_columns, axis=1, inplace=True)

重複 ['発行済株式（自己株式を除く。）の総数に対する所有株式数の割合', '連結子会社の数', '役員のうち女性の比率', '所有株式数', '議決権の数（個）', '所有議決権数（個）', '株式数（株）', '総株主の議決権に対する所有議決権数の割合', '持分法を適用した関連会社の数']


### df時点とdf期間で同じ名前の列があれば、名前に「-時点」や「-期間」を付加します。

In [342]:
v = [x for x in df時点.columns if not x in 除外columns]
for name in v:
    if name in df期間.columns:
        print('%d %d %s' % (df時点[name].notnull().sum(), df期間[name].notnull().sum(), name))
        df時点.rename(columns={ name: name+'-時点' }, errors='raise', inplace=True)                    
        df期間.rename(columns={ name: name+'-期間' }, errors='raise', inplace=True)

19332 18864 為替換算調整勘定
2437 1469 社債発行費
656 2653 株式交付費
13774 7873 繰延ヘッジ損益
8037 1752 土地再評価差額金
55100 25858 その他有価証券評価差額金
8 183 キャッシュ・フロー・ヘッジの有効部分


### 内容が同じで名前が違う2つの列を1つに統合します。

In [0]:
def unify_columns(tbl, alias, name):
    diff = 0

    # 別名と本名の列のそれぞれの値に対し
    for idx, (alias_val, val) in enumerate(zip(tbl[alias], tbl[name])):
        if not alias_val is None:
            # 別名の値がある場合

            if val is None:
                # 本名の値がない場合

                # 別名の値を本名の値とする。
                tbl[name].iloc[idx] = alias_val
            else:
                # 本名の値がある場合

                if val != 0:

                    # 別名と本名の値の違いを調べる。
                    diff = max(diff, abs((alias_val - val)/val) )

    print('%.5f %s' % (diff, alias))

In [344]:
for suffix in [ '（IFRS）', '、経営指標等', '（平成26年3月28日財規等改正後）', '又は売上総損失（△）' ]:
    for tbl in [df時点, df期間]:
        drop_names = []
        for name in tbl.columns:
            
            if  suffix in name:
                # 名前に接尾辞が含まれる場合

                # 名前から接尾辞を取り除く。
                short_name = name.replace(suffix, '')
                
                if short_name in tbl.columns:
                    # 短い名前の列がある場合

                    # 短い名前の列に統合する。
                    unify_columns(tbl, name, short_name)

                    # 長い名前の列は削除する。
                    drop_names.append(name)

                else:
                    # 短い名前の列がない場合

                    if suffix != '（IFRS）':

                        # 長い名前を短い名前に変える。
                        print('短縮 : %s -> %s' % (name, short_name))
                        tbl.rename(columns={ name:short_name }, errors='raise', inplace=True)                    

        # 長い名前の列を削除する。
        tbl.drop(drop_names, axis=1, inplace=True)

0.00000 建物及び構築物（IFRS）
0.84279 技術資産（IFRS）
0.00000 繰延税金資産（IFRS）
0.00000 のれん（IFRS）
0.00000 流動資産（IFRS）
40.36364 顧客関連資産（IFRS）
0.88627 顧客との関係（IFRS）
0.00000 ソフトウエア仮勘定（IFRS）
0.79772 製品関連無形資産（IFRS）
0.00000 繰延税金負債（IFRS）
0.00000 負債（IFRS）
0.69302 土地、建物及び構築物（IFRS）
0.00000 建設仮勘定（IFRS）
4.57535 建物及び附属設備（IFRS）
0.00000 流動負債（IFRS）
0.00000 資本金（IFRS）
5.89972 販売権（IFRS）
0.00000 商標権（IFRS）
0.00000 投資不動産（IFRS）
0.00000 機械装置及び運搬具（IFRS）
0.00000 利益剰余金（IFRS）
0.90942 のれん及び無形資産（IFRS）
0.00000 資産（IFRS）
48.67916 無形資産（IFRS）
0.00000 自己株式（IFRS）
0.00000 工具、器具及び備品（IFRS）
0.70720 器具及び備品（IFRS）
0.00000 資本剰余金（IFRS）
0.82075 機械装置（IFRS）
0.00000 引当金、流動負債（IFRS）
0.00000 特許権（IFRS）
0.00000 車両運搬具（IFRS）
39.95161 仕掛研究開発（IFRS）
0.00000 1年内返済予定の長期借入金（IFRS）
0.00000 土地（IFRS）
0.00000 有形固定資産（IFRS）
0.00000 ソフトウエア（IFRS）
0.00000 建物（IFRS）
0.00000 減価償却費、営業活動によるキャッシュ・フロー（IFRS）
0.00000 定期預金の預入による支出、投資活動によるキャッシュ・フロー（IFRS）
8.97561 営業活動によるキャッシュ・フロー（IFRS）、経営指標等
0.00000 長期借入金の返済による支出、財務活動によるキャッシュ・フロー（IFRS）
0.00000 受取利息及び受取配当金、営業活動によるキャッシュ・フロー（IFRS）
0.00000 社債の

### 長い項目名を短くします。

In [345]:
df期間.columns

Index(['EDINETコード', '会計期間終了日', 'コンテキスト',
       '受取配当金の合計額、非上場株式以外の株式、保有目的が純投資目的である投資株式、提出会社', '店舗閉鎖損失',
       'その他、その他の費用（IFRS）', '有価証券売却損', '寄付金、販売費及び一般管理費', '有価証券の売却による収入',
       '１株当たり配当額',
       ...
       '新株予約権戻入益', 'その他、営業外費用', '退職給付に係る調整額', '非支配株主に係る包括利益',
       '株式報酬費用、営業活動によるキャッシュ・フロー（IFRS）', '荷造運搬費、販売費及び一般管理費',
       '監査証明業務に基づく報酬－連結子会社', '非継続事業からの当期利益（△損失）', '投資不動産の売却による収入', '売上原価'],
      dtype='object', length=593)

In [0]:
df時点 = df時点.rename(columns={ 
    '１株当たり純資産額': '１株当たり純資産',
    '平均年齢（年）'    : '平均年齢',
    '平均勤続年数（年）': '平均勤続年数',
})

df期間 = df期間.rename(columns={ 
    # '売上総利益又は売上総損失（△）'            :'売上総利益', 
    '経常利益又は経常損失（△）'                : '経常利益', 
    '営業利益又は営業損失（△）'                : '営業利益',
    '当期純利益又は当期純損失（△）'            :'純利益', 
    '税引前当期純利益又は税引前当期純損失（△）':'税引前純利益', 
    '１株当たり当期純利益又は当期純損失（△）'  :'１株当たり純利益',
    '親会社株主に帰属する当期純利益又は親会社株主に帰属する当期純損失（△）': '親会社株主に帰属する純利益',
    '潜在株式調整後1株当たり当期純利益'         :'調整1株当たり純利益',
    '現金及び現金同等物の増減額（△は減少）'    :'現金及び現金同等物の増減' 
}, errors='raise')

### コンテキストごとに分離します。

In [0]:
df提出日時点 = df提出日時点.set_index(['EDINETコード', '会計期間終了日'])

tbls = []
for tbl, ctx in [ [df時点, '当期連結時点'], [df時点, '当期個別時点'], [df時点, '前期連結時点'], [df時点, '前期個別時点'], \
                  [df期間, '当期連結期間'], [df期間, '当期個別期間'], [df期間, '前期連結期間'], [df期間, '前期個別期間'] ]:

    tmp = tbl[tbl['コンテキスト'] == ctx]
    
    tmp = tmp.set_index(['EDINETコード', '会計期間終了日'])
    
    tmp = tmp.drop('コンテキスト', axis=1)

    tbls.append(tmp)

df当期連結時点, df当期個別時点, df前期連結時点, df前期個別時点, df当期連結期間, df当期個別期間, df前期連結期間, df前期個別期間 = tbls

assert len(df提出日時点) == len(df当期連結時点) == len(df当期個別時点) == len(df当期連結期間) == len(df当期個別期間) == len(df前期連結時点) == len(df前期個別時点) == len(df前期連結期間) == len(df前期個別期間)

### 会計終了時点と会計期間の表を連結します。

In [0]:
df当期連結 = pd.concat([ df当期連結時点, df当期連結期間 ], axis=1)
df当期個別 = pd.concat([ df当期個別時点, df当期個別期間 ], axis=1)
df前期連結 = pd.concat([ df前期連結時点, df前期連結期間 ], axis=1)
df前期個別 = pd.concat([ df前期個別時点, df前期個別期間 ], axis=1)

assert len(df当期連結) == len(df当期連結時点)  == len(df当期連結期間) == len(df当期個別) == len(df当期個別時点)  == len(df当期個別期間)
assert len(df前期連結) == len(df前期連結時点)  == len(df前期連結期間) == len(df前期個別) == len(df前期個別時点)  == len(df前期個別期間)

assert list(df当期連結.columns) == list(df当期個別.columns) == list(df前期連結.columns) == list(df前期個別.columns)

In [349]:
column_cnts = df当期連結.notnull().sum().items()
sorted_column_cnts = sorted(column_cnts, key=lambda x: x[1], reverse=True)

sorted_columns = [x[0] for x in sorted_column_cnts]

list(enumerate(sorted_column_cnts[:200]))

[(0, ('従業員数', 15131)),
 (1, ('１株当たり純資産', 14583)),
 (2, ('自己資本比率', 14580)),
 (3, ('純資産額', 14580)),
 (4, ('総資産額', 14580)),
 (5, ('１株当たり純利益', 14572)),
 (6, ('純資産', 14455)),
 (7, ('株主資本', 14454)),
 (8, ('利益剰余金', 14454)),
 (9, ('負債純資産', 14454)),
 (10, ('資産', 14454)),
 (11, ('資本金', 14454)),
 (12, ('負債', 14454)),
 (13, ('流動資産', 14450)),
 (14, ('現金及び現金同等物の残高', 14450)),
 (15, ('流動負債', 14450)),
 (16, ('固定資産', 14450)),
 (17, ('現金及び預金', 14450)),
 (18, ('現金及び現金同等物の増減', 14450)),
 (19, ('包括利益', 14450)),
 (20, ('経常利益', 14450)),
 (21, ('小計、営業活動によるキャッシュ・フロー', 14450)),
 (22, ('営業活動によるキャッシュ・フロー', 14450)),
 (23, ('投資活動によるキャッシュ・フロー', 14449)),
 (24, ('親会社株主に係る包括利益', 14448)),
 (25, ('税引前純利益', 14448)),
 (26, ('投資その他の資産', 14447)),
 (27, ('営業利益', 14446)),
 (28, ('営業外収益', 14446)),
 (29, ('法人税、住民税及び事業税', 14444)),
 (30, ('その他、流動資産', 14438)),
 (31, ('財務活動によるキャッシュ・フロー', 14430)),
 (32, ('法人税等', 14426)),
 (33, ('当期変動額合計', 14420)),
 (34, ('営業外費用', 14395)),
 (35, ('その他、流動負債', 14369)),
 (36, ('有形固定資産', 14369)),
 (37, ('固定

### 期首期末平均資産を計算します。

In [0]:
def set_assets_mean(tbl):
    # 期首期末平均資産
    mean_assets = np.ndarray(shape=(len(tbl),), dtype=np.float32)

    # EDINETコード
    edinet_codes = tbl.index.map(lambda x: x[0])
    
    prev_code = None
    prev_value = None

    # EDINETコードと資産に対し
    for i, [code, value] in enumerate(zip(edinet_codes, tbl['資産'])):
        if prev_code != code:
            # 会社の最初の場合

            # 前期がないので今期の資産を期首期末平均資産とする。
            mean_assets[i] = value

            prev_code = code
        else:
            # 会社の最初でない場合

            # 前期と今期の平均の資産を期首期末平均資産とする。
            mean_assets[i] = (prev_value + value) / 2

        # 前期の資産
        prev_value = value

    tbl['期首期末平均資産'] = mean_assets

for tbl in [ df当期連結, df当期個別, df前期連結, df前期個別 ]:
    set_assets_mean(tbl)

### 財務指標を計算します。

In [0]:
for tbl in [ df当期連結, df当期個別, df前期連結, df前期個別 ]:
    tbl['粗利益'] = tbl['売上高'] - tbl['売上原価']

    tbl['売上高総利益率']   = tbl['粗利益']   / tbl['売上高']
    tbl['売上高営業利益率'] = tbl['営業利益'] / tbl['売上高']
    tbl['売上高経常利益率'] = tbl['経常利益'] / tbl['売上高']

    tbl['売上高販管費率']   = tbl['販売費及び一般管理費'] / tbl['売上高']

    tbl['総資本回転率']     = tbl['売上高'] / tbl['資産']
    tbl['流動比率']         = tbl['流動資産'] / tbl['流動負債']

    tbl['総資産経常利益率'] = tbl['経常利益'] / tbl['期首期末平均資産']

    tbl['総資産純利益率']   = tbl['純利益']   / tbl['期首期末平均資産']

    tbl['総資産親会社株主に帰属する純利益率']   = tbl['親会社株主に帰属する純利益'] / tbl['期首期末平均資産']

    tbl['自己資本'] = tbl['株主資本'].fillna(0) + tbl['評価・換算差額等'].fillna(0)

    tbl['有利子負債'] = tbl['短期借入金'].fillna(0) + tbl['1年内返済予定の長期借入金'].fillna(0) + tbl['1年内償還予定の社債'].fillna(0) + tbl['長期借入金'].fillna(0) \
        + tbl['社債'].fillna(0) + tbl['転換社債型新株予約権付社債'].fillna(0) + tbl['コマーシャル・ペーパー'].fillna(0)

    tbl['負債比率'] = tbl['負債'] / tbl['自己資本']
    tbl['有利子負債比率'] = tbl['有利子負債']  / tbl['自己資本']

### 前期の列名に「前期」の接頭語を付けます。

In [0]:
for tbl in [ df前期連結, df前期個別 ]:
    tbl.rename(columns=dict((x, '前期' + x) for x in tbl.columns), inplace=True)

### 連結と個別のそれぞれの当期と前期の表を連結します。

In [0]:
df連結 = pd.concat([ df当期連結, df前期連結 ], axis=1)
df個別 = pd.concat([ df当期個別, df前期個別 ], axis=1)

assert len(df連結) == len(df当期連結) == len(df前期連結) == len(df個別) == len(df当期個別) == len(df前期個別)

### 当期の列名のリストを得ます。

In [0]:
assert list(df当期連結.columns) == list(df当期個別.columns)

num_columns = list(df当期連結.columns)

# 列名の重複がないことを確認
assert len(num_columns) == len(set(num_columns))

### 前期から当期への変化率を計算します。

In [0]:
for tbl in [ df連結, df個別 ]:
    for title in num_columns:
        tbl[title + '変化率'] = (tbl[title] - tbl['前期' + title]) / tbl['前期' + title]

### 前期のデータは以降で使わないので削除します。

In [0]:
drop_columns = ['前期' + x for x in num_columns]

df連結 = df連結.drop(drop_columns, axis=1)
df個別 = df個別.drop(drop_columns, axis=1)


### 個別の列名に「個別」の接頭語を付けます。

In [0]:
df個別.rename(columns=dict((x, '個別' + x) for x in df個別.columns), inplace=True)

### 連結と個別のデータを１つにまとめます。

In [0]:
df = pd.concat([ df連結, df個別 ], axis=1)

assert len(df) == len(df提出日時点)

# 列名の重複がないことを確認します。
assert len(df.columns) == len(set(df.columns))

#### 次期変化率を計算します。

In [0]:
# 予測する列の名前
y_column = target + '次期変化率'

# 次期変化率の配列をNaNで初期化します。
change_rate = np.array([np.nan] * len(df), dtype=np.float32)

# EDINETコード
edinet_codes = df.index.map(lambda x: x[0])

# 直前のEDINETコード
prev_code = None

# 直前の値
prev_value = None

for i, [code, value] in enumerate(zip(edinet_codes, df[target])):
    if prev_code != code:
        # 会社が変わった場合

        prev_code = code
    else:
        # 会社が同じ場合

        if prev_value != 0:
            # 直前の値が0でない場合

            # 次期変化率を計算します。
            change_rate[i - 1] = (value - prev_value) / prev_value

    prev_value = value

df[y_column] = change_rate

### 次期変化率が欠損値の行を削除します。

In [0]:
df = df.dropna(subset=[y_column])

### 比率以外の列は財務分析に使わないので削除します。

In [0]:
df = df.drop([x for x in df.columns if not x.endswith('率')], axis=1)

### 会社情報をセットします。

#### 会社情報のCSVを読み込みます。

In [0]:
df会社 = pd.read_csv("EdinetcodeDlInfo.csv", encoding='cp932', skiprows=[0])
df会社 = df会社.set_index('ＥＤＩＮＥＴコード')

#### 会社名から "株式会社" を取り除きます。

In [0]:
df会社['提出者名'] = df会社['提出者名'].apply(lambda x: x.replace('株式会社', '').strip())

#### 会社名と業種をセットします。

In [0]:
edinet_codes = df.index.map(lambda x: x[0])
df['会社名']  = edinet_codes.map(lambda x: df会社.loc[x]['提出者名'])
df['業種']    = edinet_codes.map(lambda x: df会社.loc[x]['提出者業種'])

In [0]:
df = df.reset_index()
df = df.set_index(['EDINETコード', '会社名', '会計期間終了日'])

### 出現頻度の高い順に列名を並べます。

In [366]:
# 有効値が多い列を得ます。
column_cnts = df.notnull().sum().items()
freq_column_cnts = sorted(column_cnts, key=lambda x: x[1], reverse=True)

list(enumerate(freq_column_cnts[:10]))

[(0, ('売上高営業利益率', 10954)),
 (1, ('売上高経常利益率', 10954)),
 (2, ('現金及び現金同等物の残高変化率', 10954)),
 (3, ('純資産変化率', 10954)),
 (4, ('個別総資産経常利益率', 10954)),
 (5, ('個別負債比率', 10954)),
 (6, ('個別有利子負債比率', 10954)),
 (7, ('売上高次期変化率', 10954)),
 (8, ('業種', 10954)),
 (9, ('総資本回転率', 10953))]

### 出現頻度の低い列は削除します。

In [0]:
drop_names = [x[0] for x in freq_column_cnts][200:]
df = df.drop(drop_names, axis=1)

### 無限大はNaNに置換します。

In [0]:
for name in [x for x in df.columns if x != '業種']:
    df[name] = df[name].replace([np.inf, -np.inf], np.nan)

### 平均0、標準偏差1に正規化します。
### 欠損値は0で置き換えます。

In [0]:
def norm(v):
    # 平均
    mean = v.mean()

    # 標準偏差
    std  = v.std()

    assert not( np.isnan(mean) or np.isnan(std) or std == 0 )

    w = (v - mean) / std

    # 欠損値は0で置き換えます。
    return w.fillna(0)

In [0]:
for name in [x for x in df.columns if x != '業種']:
    df[name] = norm(df[name])

### 外れ値をクリップします。

In [0]:
nrow, ncol = (5, 4)

names = [x[0] for x in freq_column_cnts if x[0] != '業種']

for label in names[: nrow*ncol]:
    tmp = df[label]
    df[label] = tmp.clip(lower=tmp.quantile(0.05), upper=tmp.quantile(0.95))


### 相関係数を計算します。

In [0]:
name_corrs = [ [name, np.corrcoef( np.stack([df[y_column].values, df[name].values]) )[0, 1] ] for name in df.columns if not name in ['業種', y_column] ]

# 欠損値は削除します。
name_corrs = [ x for x in name_corrs if not np.isnan(x[1])]

### 相関係数の絶対値の大きい順にソートします。

In [373]:
sorted_name_corrs = sorted(name_corrs, key=lambda x: abs(x[1]), reverse=True)
['%d  %.5f %s' % (i, x[1], x[0]) for i, x in enumerate(sorted_name_corrs[:50]) ]

['0  0.27654 負債純資産変化率',
 '1  0.27654 資産変化率',
 '2  0.27571 総資産額変化率',
 '3  0.26525 期首期末平均資産変化率',
 '4  0.24844 個別負債純資産変化率',
 '5  0.24844 個別資産変化率',
 '6  0.24823 個別総資産額変化率',
 '7  0.24014 純資産変化率',
 '8  0.23770 流動資産変化率',
 '9  0.23382 個別期首期末平均資産変化率',
 '10  0.22250 自己資本変化率',
 '11  0.19972 個別純資産変化率',
 '12  0.19896 個別自己資本変化率',
 '13  0.19458 販売費及び一般管理費変化率',
 '14  0.19163 従業員数変化率',
 '15  0.15754 流動負債変化率',
 '16  0.15160 個別固定資産変化率',
 '17  0.15068 個別総資産経常利益率',
 '18  0.14433 負債変化率',
 '19  0.14202 売上高営業利益率',
 '20  0.14072 個別総資産純利益率',
 '21  0.13559 売上高経常利益率',
 '22  0.12982 自己資本利益率',
 '23  0.12673 固定資産変化率',
 '24  0.11833 総資産経常利益率',
 '25  0.11729 個別純資産額変化率',
 '26  0.11613 売上高総利益率',
 '27  0.11193 受取手形及び売掛金変化率',
 '28  0.10492 個別利益剰余金変化率',
 '29  0.09307 個別売上高総利益率',
 '30  0.09224 現金及び現金同等物の残高変化率',
 '31  0.09005 売上高変化率',
 '32  0.08386 純資産額変化率',
 '33  0.08009 その他、流動資産変化率',
 '34  0.07729 個別流動比率',
 '35  0.07258 営業外収益変化率',
 '36  0.07222 個別投資その他の資産変化率',
 '37  0.06769 １株当たり純資産変化率',
 '38  0.06605 減価償却費、営業活動によるキャッシュ・フロ

### 相関係数が低い列を削除します。

In [0]:
sorted_names = [x[0] for x in sorted_name_corrs]
drop_names = sorted_names[50:] 
df = df.drop(drop_names, axis=1)


### 業種をダミー変数に変換します。

In [0]:
df = pd.get_dummies(df)

# '業種'の列がなくなったことを確認
assert not '業種' in df.columns

### pickleでファイルに書きます。

In [0]:
import pickle

data = { 'y_column': y_column, 'data_frame': df  }

if   y_column == '売上高次期変化率':
    file_name = 'preprocess-uriage.pickle'
elif y_column == '営業利益次期変化率':
    file_name = 'preprocess-eigyo.pickle'
elif y_column == '経常利益次期変化率':
    file_name = 'preprocess-keijo.pickle'
elif y_column == '税引前純利益次期変化率':
    file_name = 'preprocess-jun.pickle'
else:
    assert False

with open(file_name, 'wb') as f:
    pickle.dump(data, f)

### パソコンにファイルをダウンロードする場合は以下のコメントをはずしてください。

In [0]:
# from google.colab import files

# files.download("preprocess.pickle")